In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
%cd /gdrive/My Drive/Thesis_Resources/Use_cases/Dead_sea_data_retrieval

/gdrive/My Drive/Thesis_Resources/Use_cases/MDPI_Pollution


In [ ]:
!pip install hda -U cdsapi

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for hda: filename=hda-1.15-py3-none-any.whl size=13928 sha256=22edc302b4c73f516a1f52cdc3c18c4c0a49cdb1c0016f5d403f509e29dfdbe6
  Stored in directory: /root/.cache/pip/wheels/fc/f4/c1/3966e0fc4c89b122365b9461c257a2d39fb4ea604227261b5b
  Created wheel for cdsapi: filename=cdsapi-0.6.1-py2.py3-none-any.whl size=12008 sha256=5be9e91c3c40696e7f35c5dae177889e09c2e97e125b430ab6ffdcb17ef2893f
  Stored in directory: /root/.cache/pip/wheels/7c/63/08/45461d6f6636c1aba7846828d8c787a064073945048f76d44a
Successfully built hda cdsapi


In [ ]:
from hda import Client, Configuration

# Configure user's credentials without a .hdarc
conf = Configuration(user = "user", password = "password")
hda_client = Client(config = conf)

In [ ]:
# The JSON query loaded in the "query" variable
query = {
  "datasetId": "EO:ESA:DAT:SENTINEL-5P:TROPOMI",
  "boundingBoxValues": [
    {
      "name": "bbox",
      "bbox": [
        8.719077057317854,
        44.39974795031061,
        11.876735035263849,
        45.57510265306455
      ]
    }
  ],
  "dateRangeSelectValues": [
    {
      "name": "position",
      "start": "2019-01-01T00:00:00.000Z",
      "end": "2022-09-21T00:00:00.000Z"
    }
  ]
}

# Ask the result for the query passed in parameter
matches = hda_client.search(query)

# List the results
print(matches)

# Download results in a directory (e.g. '/tmp')
# Create the directory if it doesn't exist since version 1.14 of hda
matches.download(download_dir="/tmp")

In [ ]:
aqicn_key = "8612d6b5d65645ad7880fce505cfb09683d60e53"

In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta

def fetch_data(city, token, start_date, end_date):
    base_url = "https://api.waqi.info/feed/{}/?token={}"
    date_range = pd.date_range(start=start_date, end=end_date)
    data_list = []

    for date in date_range:
        response = requests.get(base_url.format(city, token))
        if response.status_code == 200:
            data = response.json()
            if data['status'] == 'ok':
                record = {
                    'date': date.strftime('%Y-%m-%d'),
                    'aqi': data['data']['aqi'],
                    'pm10': data['data']['iaqi'].get('pm10', {}).get('v'),
                    'pm25': data['data']['iaqi'].get('pm25', {}).get('v'),
                    'o3': data['data']['iaqi'].get('o3', {}).get('v'),
                    # ... you can add more pollutants as needed
                }
                data_list.append(record)
        else:
            print(f"Failed to fetch data for date: {date}")

    return pd.DataFrame(data_list)

# Define the parameters
TOKEN = aqicn_key
CITY = 'milan'
START_DATE = (datetime.now() - timedelta(days=365)).strftime('%Y-%m-%d')  # One year ago from today
END_DATE = datetime.now().strftime('%Y-%m-%d')  # Today

# Fetch the data
df = fetch_data(CITY, TOKEN, START_DATE, END_DATE)

# Save the data to a CSV file
df.to_csv('milan_air_quality_last_year.csv', index=False)


In [ ]:
df

,date,aqi,pm10,pm25,o3
0,2022-10-06,30,11,30,None
1,2022-10-07,30,11,30,None
2,2022-10-08,30,11,30,None
3,2022-10-09,30,11,30,None
4,2022-10-10,30,11,30,None
...,...,...,...,...,...
361,2023-10-02,30,11,30,None
362,2023-10-03,30,11,30,None
363,2023-10-04,30,11,30,None
364,2023-10-05,30,11,30,None
